In [18]:
from Models import *
from Utils import *
from FastModels import *
import warnings
from livelossplot import PlotLossesKeras
# warnings.filterwarnings('ignore')

In [19]:
from gensim.models import KeyedVectors
import sentencepiece as spm

max_len = 15
enablePadding = True

sp = spm.SentencePieceProcessor()
sp.Load('/work/data/bpe/en.wiki.bpe.op50000.model')
bpe = KeyedVectors.load_word2vec_format("/work/data/bpe/en.wiki.bpe.op50000.d200.w2v.bin", binary=True)
bpe.index2word = [''] + bpe.index2word + ['<sos>'] + ['<eos>']  # add empty string 
nb_words = len(bpe.index2word)
# word2index
bpe_dict = {bpe.index2word[i]: i for i in range(len(bpe.index2word))}
# construct embedding_matrix
embedding_matrix = np.concatenate([np.zeros((1, bpe.vector_size)), bpe.vectors, np.zeros((2, bpe.vector_size))]) # add zero vector for empty string (i.e. used for padding)

embedding_layer = Embedding(nb_words,
                    embedding_matrix.shape[-1],
                    weights=[embedding_matrix],
                    input_length=max_len,
                    trainable=True)

In [20]:
enablePadding = True
df_may, qrel_may = get_test_data("MayFlower", "/work/")
df_june, qrel_june = get_test_data("JuneFlower", "/work/")
df_july, qrel_july = get_test_data("JulyFlower", "/work/")

q_may = parse_texts_bpe(df_may.q.tolist(), sp, bpe_dict, max_len, enablePadding, "post")
d_may = parse_texts_bpe(df_may.d.tolist(), sp, bpe_dict, max_len, enablePadding, "post")

q_june = parse_texts_bpe(df_june.q.tolist(), sp, bpe_dict, max_len, enablePadding, "post")
d_june = parse_texts_bpe(df_june.d.tolist(), sp, bpe_dict, max_len, enablePadding, "post")

q_july = parse_texts_bpe(df_july.q.tolist(), sp, bpe_dict, max_len, enablePadding, "post")
d_july = parse_texts_bpe(df_july.d.tolist(), sp, bpe_dict, max_len, enablePadding, "post")

test_set = [[q_may, d_may, qrel_may, df_may, "MayFlower"], [q_june, d_june, qrel_june, df_june, "JuneFlower"], [q_july, d_july, qrel_july, df_july, "JulyFlower"]]


b'Skipping line 19898: expected 6 fields, saw 8\nSkipping line 20620: expected 6 fields, saw 8\nSkipping line 38039: expected 6 fields, saw 8\n'


In [205]:
df = pd.read_csv("/work/data/train_data/30M_QD_lower2.txt", nrows=200000, names=["label", "q", "d"], sep="\t", header=None, error_bad_lines=False)

In [4]:
enablePadding = False
q_df = parse_texts_bpe(df_july.q.tolist(), sp, bpe_dict, max_len, enablePadding, "post")
d_df = parse_texts_bpe(df_july.d.tolist(), sp, bpe_dict, max_len, enablePadding, "post")


In [5]:
def addTags(x, bpe_dict, max_len):
    dec_input = np.copy(x)
    dec_output = np.copy(x)
    for i in range(len(x)):
        dec_input[i] = [bpe_dict['<sos>']] + dec_input[i] + [bpe_dict['<eos>']]
        dec_output[i] = dec_output[i] + [bpe_dict['<eos>']]

    return pad_sequences(dec_input, maxlen=max_len, padding="post"), pad_sequences(dec_output, maxlen=max_len, padding="post")

In [6]:
dec_inputs, dec_outputs = addTags(q_df, bpe_dict, max_len)

In [7]:
enablePadding = True
q_df = parse_texts_bpe(df_july.q.tolist(), sp, bpe_dict, max_len, enablePadding, "post")
d_df = parse_texts_bpe(df_july.d.tolist(), sp, bpe_dict, max_len, enablePadding, "post")


In [209]:
latent_dim = 200

# Define an input sequence and process it.
encoder_inputs = Input(shape=(max_len,))
encoder_embedding = Embedding(nb_words,
                                embedding_matrix.shape[-1],
                                weights=[embedding_matrix],
                                input_length=max_len,
                                mask_zero=True,
                                trainable=True)

encoder_lstm = LSTM(latent_dim, return_state=True)

x = BatchNormalization()(encoder_embedding(encoder_inputs))
state, state_h, state_c = encoder_lstm(x)

mean = Dense(latent_dim)
var = Dense(latent_dim)

state_mean = mean(state)
state_var = var(state)

# state_h_mean = mean(state_h)
# state_h_var = var(state_h)

# state_c_mean = mean(state_c)
# state_c_var = mean(state_c)

def sampling(args):
    z_mean, z_log_var = args
#     epsilon = K.random_normal(shape=(K.shape(z_mean)[0], K.shape(z_mean)[1]), mean=0.,\
#                               stddev=1)
#     return z_mean + K.exp(z_log_var / 2) * epsilon 
    return z_mean + z_log_var

# state_h_z = Lambda(sampling)([state_h_mean, state_h_var])
# state_c_z = Lambda(sampling)([state_c_mean, state_c_var])
state_z = Lambda(sampling)([state_mean, state_var])


# encoder_states = [state_h_z, state_c_z]
# encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
# decoder_inputs = Input(shape=(max_len,))
# decoder_embedding = Embedding(nb_words,
#                                 embedding_matrix.shape[-1],
#                                 weights=[embedding_matrix],
#                                 input_length=max_len,
#                                 mask_zero=True,
#                                 trainable=True)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_dense = Dense(nb_words, activation='softmax')
x = encoder_embedding(decoder_inputs)

# decoder_outputs, _, _ = decoder_lstm(x, initial_state=encoder_states)
decoder_outputs, _, _ = decoder_lstm(RepeatVector(max_len)(state_z))

rec_outputs = decoder_dense(decoder_outputs)
kl_outputs = decoder_dense(decoder_outputs)


def kl_loss(x, x_):
    kl_loss = - 0.5 * K.sum(1 + state_var - K.square(state_mean) - K.exp(state_var), axis=-1)
    return kl_loss

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs], [rec_outputs, kl_outputs])
model.compile("rmsprop", ["sparse_categorical_crossentropy", kl_loss])


In [26]:
# # query_encoder = Model(encoder_inputs, [state_h_mean, state_c_mean, state_h_z, state_c_z])
# # query_encoder = Model(encoder_inputs, [state_h, state_c])
# query_encoder = Model(encoder_inputs, state_mean)

# encoder_model = Model(encoder_inputs, encoder_states)

# decoder_state_input_h = Input(shape=(latent_dim,))
# decoder_state_input_c = Input(shape=(latent_dim,))
# decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding(decoder_inputs), initial_state=decoder_states_inputs)

# decoder_states = [state_h, state_c]
# decoder_outputs = decoder_dense(decoder_outputs)
# decoder_model = Model(
#     [decoder_inputs] + decoder_states_inputs,
#     [decoder_outputs] + decoder_states)


In [145]:
enablePadding = False
q_july = parse_texts_bpe(df_july.q.tolist(), sp, bpe_dict, max_len, enablePadding, "post")
d_july = parse_texts_bpe(df_july.d.tolist(), sp, bpe_dict, max_len, enablePadding, "post")

def addTags(x, bpe_dict, max_len):
    dec_input = np.copy(x)
    dec_output = np.copy(x)
    for i in range(len(x)):
        dec_input[i] = [bpe_dict['<sos>']] + dec_input[i] + [bpe_dict['<eos>']]
        dec_output[i] = dec_output[i] + [bpe_dict['<eos>']]

    return pad_sequences(dec_input, maxlen=max_len, padding="post"), pad_sequences(dec_output, maxlen=max_len, padding="post")

q_dec_inputs, q_dec_outputs = addTags(q_july, bpe_dict, max_len)
d_dec_inputs, d_dec_outputs = addTags(d_july, bpe_dict, max_len)


enablePadding = True
q_july = parse_texts_bpe(df_july.q.tolist(), sp, bpe_dict, max_len, enablePadding, "post")
d_july = parse_texts_bpe(df_july.d.tolist(), sp, bpe_dict, max_len, enablePadding, "post")


In [6]:
from sklearn.metrics.pairwise import euclidean_distances
from scipy.stats import wasserstein_distance
def eval_july(run, idx=None):
    q_ = run.predict(q_july)[idx] if idx != None else run.predict(q_july)
    d_ = run.predict(d_july)[idx] if idx != None else run.predict(d_july)
    cosine = CosineSim(q_.shape[-1])
    
    pred = []
    for i in range(len(q_)):
        pred.append(wasserstein_distance(q_[i], d_[i]))
    pred = np.array(pred)
    cos_pred = cosine.model.predict([q_, d_])
    return auc(qrel_july, pred.flatten()), auc(qrel_july, cos_pred.flatten())

In [211]:
print(eval_july(query_encoder))
# print(eval_july(query_encoder, 1))    
# print(eval_july(query_encoder, 2))    
# print(eval_july(query_encoder, 3))  

(0.4137739331539505, 0.5884498209877915)


In [204]:
print(eval_july(w2v))

(0.5466432431230404, 0.5306766883853322)


In [212]:
tmp = np.expand_dims(q_dec_outputs, axis=-1)
tmp2 = np.expand_dims(d_dec_outputs, axis=-1)
# tmp = np.expand_dims(dec_outputs, axis=-1)

x_train = [np.concatenate([q_july, d_july])]
y_train = [np.concatenate([tmp2, tmp]), np.concatenate([tmp2, tmp])]
for i in range(10):
    model.fit(x_train, y_train,
              batch_size=32,
              epochs=1,
              validation_split=0.2,
              verbose=2)
    #           validation_split=0.2, callbacks=[TQDMNotebookCallback()])
    #     print(eval_july(query_encoder, 0))
    #     print(eval_july(query_encoder, 1))    
    #     print(eval_july(query_encoder, 2))    
    #     print(eval_july(query_encoder, 3))    
    print(eval_july(query_encoder))    

Train on 681 samples, validate on 171 samples
Epoch 1/1
 - 10s - loss: 11.3695 - dense_39_loss_1: 6.3068 - dense_39_loss_2: 5.0627 - val_loss: 3.4706 - val_dense_39_loss_1: 3.0098 - val_dense_39_loss_2: 0.4608
(0.45864929173430585, 0.5846249638639952)
Train on 681 samples, validate on 171 samples
Epoch 1/1
 - 3s - loss: 4.3326 - dense_39_loss_1: 2.9347 - dense_39_loss_2: 1.3980 - val_loss: 3.0858 - val_dense_39_loss_1: 2.7424 - val_dense_39_loss_2: 0.3434
(0.4755720607529632, 0.5847361515710823)
Train on 681 samples, validate on 171 samples
Epoch 1/1
 - 3s - loss: 3.2853 - dense_39_loss_1: 2.7304 - dense_39_loss_2: 0.5548 - val_loss: 2.8686 - val_dense_39_loss_1: 2.6103 - val_dense_39_loss_2: 0.2584
(0.4634970757633036, 0.5747737330160778)
Train on 681 samples, validate on 171 samples
Epoch 1/1
 - 3s - loss: 2.8282 - dense_39_loss_1: 2.5856 - dense_39_loss_2: 0.2426 - val_loss: 2.7107 - val_dense_39_loss_1: 2.4959 - val_dense_39_loss_2: 0.2149
(0.4768396006137561, 0.5726166914985878)
T

In [ ]:
print(eval_july(query_encoder))

In [7]:
w2v = Sequential([Embedding(nb_words,
                    embedding_matrix.shape[-1],
                    weights=[embedding_matrix],
                    input_length=max_len,
                    trainable=True), GlobalAveragePooling1D()])
eval_july(w2v)

(0.5466432431230404, 0.530643332073206)

In [10]:
test_set = []
path = "/work/"
for i in ["MayFlower", "JuneFlower", "JulyFlower", "sts", "quora", "para"]:
    df, qrel = get_test_data(i, path)
    q_ = parse_texts_bpe(df.q.tolist(), sp, bpe_dict, max_len, enablePadding)
    d_ = parse_texts_bpe(df.d.tolist(), sp, bpe_dict, max_len, enablePadding)
    test_set.append([q_, d_, qrel, df, i])

b'Skipping line 19898: expected 6 fields, saw 8\nSkipping line 20620: expected 6 fields, saw 8\nSkipping line 38039: expected 6 fields, saw 8\n'


In [13]:
evaluate(w2v, test_set)

(0.5133590915830876,
 0.8435006560138728,
 0.5306544508439147,
 0.7403234703484198,
 0.93896937380083,
 0.4022813698080795)

In [95]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value, _, _ = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, max_len))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = bpe_dict['<sos>']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h = decoder_model.predict([target_seq, states_value])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = bpe.index2word[sampled_token_index]
        if sampled_char == '':
            stop_condition = True
            
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '<eos>' or
           len(decoded_sentence) > max_len):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, max_len))
        target_seq[0, 0 ] = sampled_token_index

        # Update states
#         states_value = [h]
        states_value = h

    return decoded_sentence

In [161]:
def generate_output(model, bpe, x, nrows=100000, idx=None):

    gen_x = np.argmax(model.predict(x), axis=-1) if idx == None else np.argmax(model.predict(x)[idx], axis=-1)
    bleu = []
    results = []
    count = 0
    for i, k in zip(gen_x, x[0]):
        gen_x = " ".join([bpe.index2word[t] for t in i])
        real_x = " ".join([bpe.index2word[t] for t in k])

        bleu.append(sentence_bleu(real_x, gen_x))
        
        real_x = real_x.replace("▁the", "")
        real_x = real_x.replace("▁","")
        gen_x = gen_x.replace("▁the", "")
        gen_x = gen_x.replace("▁","")
        
        if count > nrows:
            break
        count = count + 1
        results.append("%s\t%s" % (real_x, gen_x))
        
    
    print("BLEU: %.4f" % np.mean(bleu))
    idx = np.arange(len(results))
    shuffle(idx)
    for i in idx[:nrows]:
        print(results[i])

In [200]:
generate_output(model, bpe, [q_july], idx=0)

BLEU: 0.6894
benedict c umber bat ch               	benedict c umber bat ch sm <eos>             
zy v ox patient assistance program              	birmingham services assistance ing <eos>               
employee services portal                 	employee services associate salary <eos>               
crime rate in su w anee ga             	crime rate in con yers ga <eos>             
lazy boy furniture                 	what is is lazy <eos> <eos>              
yellow teeth to white teeth               	how to  to of  <eos> <eos>            
where is james best buried               	where is james buried buried <eos>              
tallest water slide in  world              	hepatitis college of wage <eos> <eos>              
adam                   	alec nem ec team <eos>               
what does you triggered me mean in spanish            	what is f mean mean <eos> <eos>             
andy warhol museum                 	radio warhol museum <eos>                
how much do auto dealership

In [96]:
for i, real in zip(q_july, df_july.q.tolist()):
    print(decode_sequence(np.array([i])), real)

▁the▁cause▁of▁hives<eos> hives causes
▁what▁is▁the▁difference difference between whey protein and whey protein isolate
▁how▁much▁does▁kybella average cost of kybella
▁macular▁hole▁surgery macular hole surgery recovery
▁where▁is▁kristen▁powers kirstin powers
▁what▁is▁the▁top▁rate top capital gains tax rate 2017
▁periadic▁table<eos> the peroidic table
▁warren▁buffett▁net▁worth warren buffett net worth 2014
▁how▁long▁spaghetti▁sauce how long spaghetti sauce in refrigerator
▁how▁to▁get▁rid▁of▁your how do they make cynder blocks
▁what▁is▁the▁books▁that rick riordan books
▁causes▁of▁colon▁cancer colon cancer symptoms in women
▁what▁is▁the▁best▁treatment jellyfish sting treatment
▁american▁horror▁story american horror story season 4
▁what▁nationality▁is▁brigitte act for america brigitte gabriel
▁20<unk>▁miss▁america 2015 miss america competition
▁how▁to▁get▁rid▁of▁pimp how to get rid of pimples
▁when▁should▁i▁plant▁my when to plant daffodil bulbs
▁how▁do▁you▁grill▁salmon how do you grill salm

KeyboardInterrupt: 